# Index Scoring
This notebook outlines how to create a heat map that represents a composite index score. 


# Methodology
Composite index scoring is typically used by understanding how a student is perfoming against their peers. [Example](https://medium.com/analytics-vidhya/the-factor-analysis-for-constructing-a-composite-index-2496686fc54c). GIS experts use this to have visuals by geography or trade area ranking and answer questions into things related to site selection, site planning, or targeting consumers.

# What is a Z-Score
Simply put, a z-score (also called a standard score or an index score) reveals how far above or below average this instance is compared to it's peers. But more technically it’s a measure of how many standard deviations below or above the mean a raw score is. More info [here](https://www.statisticshowto.com/probability-and-statistics/z-score/). 



In [ ]:
import pandas as pd
!pip install geopandas
import geopandas as gpd

import numpy as np 
from sklearn import preprocessing


     |████████████████████████████████| 1.0 MB 4.9 MB/s 
     |████████████████████████████████| 16.7 MB 48.1 MB/s 
     |████████████████████████████████| 6.3 MB 28.0 MB/s 


In [ ]:
# # Pandas Read CSV
df = pd.read_csv("/content/node_composite_score3.csv", encoding_errors='ignore')


# # you could drop rows where our variables all = 0 it will create holes in our final gdf
df2 = df[['FIPS','Within_DTPlus_TTA','Income_Attainment', 'Edu_Attainment', 'Eth_Hispanic_CrYr','geometry']]

df3 = df[['FIPS','Income_Attainment', 'Edu_Attainment', 'Eth_Hispanic_CrYr']]


# round our values
df3.head(3)

,FIPS,Income_Attainment,Edu_Attainment,Eth_Hispanic_CrYr
0,480019501001,340,729,127
1,480019501002,312,1035,190
2,480019501003,355,733,106


### Standardizing Each Variable

This is important if you have to compare apples to oranges. Meaning, you might have a vairiable that's a percentage while another variable that's a raw count. 


### Why -3 to 3?
Defining the ranges in the minmax scaler from -3 to 3. 
I want to have enough variance and a typical 0-1 variance is a bit tight.
> In most large data sets, 99% of values have a Z-score between -3 and 3, meaning they lie within three standard deviations above or below the mean.
- [Investopia Team](https://www.investopedia.com/ask/answers/021115/what-difference-between-standard-deviation-and-z-score.asp)




In [ ]:
## define the scaler
## scaler is set to a range of -3 to 3 
scaler = preprocessing.MinMaxScaler(feature_range=(-3, 3), copy=True, clip=False)


## Testing with a seperate df to keep my sanity
df_scaled = df3.copy()

## Run the minmax scaler on our target variables
df_scaled[['Income_Attainment', 'Edu_Attainment','Eth_Hispanic_CrYr']] = scaler.fit_transform(df_scaled[['Eth_Hispanic_CrYr', 'Income_Attainment','Edu_Attainment']]).round(2)

## rename our columns to a human-friendly readable column
df_scaled.rename(columns = {'Income_Attainment':'Income_Attainment_score', 'Edu_Attainment':'Edu_Attainment_score', 'Eth_Hispanic_CrYr':'Eth Hispanic CrYr_score'}, inplace = True)

 
df_scaled = df_scaled[['FIPS', 'Income_Attainment_score','Edu_Attainment_score','Eth Hispanic CrYr_score' ]]
df_scaled

,FIPS,Income_Attainment_score,Edu_Attainment_score,Eth Hispanic CrYr_score
0,480019501001,-2.94,-2.64,-2.77
1,480019501002,-2.92,-2.67,-2.68
2,480019501003,-2.95,-2.63,-2.77
3,480019504011,-2.41,-2.96,-1.70
4,480019504021,-2.24,-2.98,-0.95
...,...,...,...,...
20815,484599501002,-2.98,-2.62,-2.77
20816,484599501003,-2.98,-2.71,-2.74
20817,484599501005,-2.92,-2.70,-2.79
20818,484599501006,-2.98,-2.68,-2.71


### Weighting

Weighting would be applied here. It's as simple as multiplying each cap with the weight value.

example: if the weight is 1/2 important - we would multiple the variable by 0.5

### Composite Variable
Here's the magic number - it's really simple. Just add each column into one.

### Final Z Score
A Z-score is a numerical measurement that describes a value's relationship to the mean of a group of values. Z-score is measured in terms of standard deviations from the mean. If a Z-score is 100, it indicates that the data point's score is identical to the mean score



In [ ]:
## Create a composite score
df_scaled['Composite_Score'] = df_scaled[["Income_Attainment_score" , "Edu_Attainment_score",'Eth Hispanic CrYr_score']].sum(axis = 1)

df_scaled[['Composite_Score']] = preprocessing.scale(df_scaled[['Composite_Score']])


df_scaled

,FIPS,Income_Attainment_score,Edu_Attainment_score,Eth Hispanic CrYr_score,Composite_Score
0,480019501001,-2.94,-2.64,-2.77,-0.318869
1,480019501002,-2.92,-2.67,-2.68,-0.200558
2,480019501003,-2.95,-2.63,-2.77,-0.318869
3,480019504011,-2.41,-2.96,-1.70,1.574096
4,480019504021,-2.24,-2.98,-0.95,2.905086
...,...,...,...,...,...
20815,484599501002,-2.98,-2.62,-2.77,-0.348446
20816,484599501003,-2.98,-2.71,-2.74,-0.437179
20817,484599501005,-2.92,-2.70,-2.79,-0.407601
20818,484599501006,-2.98,-2.68,-2.71,-0.348446


In [ ]:


df_scaled[['Composite_Score']] = scaler.fit_transform(df_scaled[['Composite_Score']]).round(2)

standscaler = preprocessing.StandardScaler()


## Define the max of the composite score
max = df_scaled['Composite_Score'].max()


## Calculate the index, results will yield a final score from 0 to 200 (100 = average)
df_scaled[['Index_Score']] = df_scaled[['Composite_Score']].apply(lambda x: (1 + (x/max))*100).round(1)
df_scaled[["Income_Attainment_score" , "Edu_Attainment_score",'Eth Hispanic CrYr_score']] = df_scaled[["Income_Attainment_score" , "Edu_Attainment_score",'Eth Hispanic CrYr_score']].apply(lambda x: (1 + (x/3))*100).round(1)


In [ ]:
## appending our score to our OG dataframe
finaldf = pd.merge(df2, df_scaled[["FIPS", "Income_Attainment_score","Edu_Attainment_score","Eth Hispanic CrYr_score","Composite_Score","Index_Score"]], on='FIPS', how='left')

## Rank field can be helpful if we were exploring geographies with less ambigous names (example top 25 CBSAs and exploring the results through table view)
finaldf['Rank'] = finaldf['Composite_Score'].rank(ascending = False)
finaldf = finaldf.sort_values(by = 'Rank').reset_index()


finaldf.head(25)

,index,FIPS,Within_DTPlus_TTA,Income_Attainment,Edu_Attainment,Eth_Hispanic_CrYr,geometry,Income_Attainment_score,Edu_Attainment_score,Eth Hispanic CrYr_score,Composite_Score,Index_Score,Rank
0,25807,481576729001,False,5745,19434,13465,"POLYGON ((-95.77548 29.70058, -95.77547 29.700...",196.0,200.0,200.0,3.00,200.0,1.0
1,27357,482015430031,False,2821,9361,13610,"POLYGON ((-95.72389 29.88066, -95.72388 29.880...",198.0,98.3,96.3,0.95,131.7,2.0
2,6334,481410103421,True,2116,5088,13742,"MULTIPOLYGON (((-106.25815 31.70097, -106.2566...",200.0,73.7,52.3,0.28,109.3,5.0
3,23046,482150242012,False,2327,6003,12567,"POLYGON ((-98.56824 26.29362, -98.56826 26.293...",183.0,81.0,61.7,0.28,109.3,5.0
4,20214,481410103421,False,2116,5088,13742,"POLYGON ((-106.28403 31.70147, -106.28413 31.7...",200.0,73.7,52.3,0.28,109.3,5.0
5,20215,481410103421,False,2116,5088,13742,"POLYGON ((-106.28403 31.70147, -106.28413 31.7...",200.0,73.7,52.3,0.28,109.3,5.0
6,6335,481410103421,True,2116,5088,13742,"MULTIPOLYGON (((-106.25815 31.70097, -106.2566...",200.0,73.7,52.3,0.28,109.3,5.0
7,25815,481576731011,False,1919,7831,11889,"POLYGON ((-95.85388 29.77770, -95.85457 29.777...",173.0,66.7,80.7,0.22,107.3,8.0
8,23759,480610125042,False,1847,4912,12563,"POLYGON ((-97.54416 25.94498, -97.54421 25.944...",183.0,64.3,50.7,0.00,100.0,9.0
9,27566,480396606021,False,1889,7698,7188,"POLYGON ((-95.44612 29.49964, -95.44615 29.499...",104.7,65.7,79.3,-0.49,83.7,10.0


In [ ]:
## export if you wanna poke around
finaldf.to_csv('node_composite_score.csv')